In [ ]:
# python notebook for Make Your Own Neural Network
# code for a 3-layer neural network, and code for learning the MNIST dataset
# this version trains using the MNIST dataset, then tests on our own images
# (c) Tariq Rashid, 2016
# license is GPLv2

## Import der notwendigen Bibliotheken

In [ ]:
import numpy
# scipy.special for the sigmoid function expit()
import scipy.special
# library for plotting arrays
import matplotlib.pyplot
# ensure the plots are inside this notebook, not an external window
%matplotlib inline

In [ ]:
# helper to load data from PNG image files
import imageio.v2 as imageio
# glob helps select multiple files using patterns
import glob

## Python-Klasse für ein neuronales Netz

**3-schichtiges Neuronales Netz**

Parameter:
- Anzahl Neuronen des Input Layer
- Anzahl Neuronen des Hidden Layer
- Anzahl Neuronen des Output-Layer
- Lernrate

In [ ]:
# neural network class definition
class neuralNetwork:
    
    
    # initialise the neural network
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # set number of nodes in each input, hidden, output layer
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        # link weight matrices, wih and who
        # weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
        # w11 w21
        # w12 w22 etc 
        self.wih = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.onodes, self.hnodes))

        # learning rate
        self.lr = learningrate
        
        # activation function is the sigmoid function
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass

    
    # train the neural network
    def train(self, inputs_list, targets_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        # output layer error is the (target - actual)
        output_errors = targets - final_outputs
        # hidden layer error is the output_errors, split by weights, recombined at hidden nodes
        hidden_errors = numpy.dot(self.who.T, output_errors) 
        
        # update the weights for the links between the hidden and output layers
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        
        # update the weights for the links between the input and hidden layers
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        
        pass

    
    # query the neural network
    def query(self, inputs_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

## Parameter setzen und KNN aufbauen

In [ ]:
# number of input, hidden and output nodes
input_nodes = 784
hidden_nodes = 200
output_nodes = 10

# learning rate
learning_rate = 0.1

# create instance of neural network
n = neuralNetwork(input_nodes,hidden_nodes,output_nodes, learning_rate)

## Trainingsdaten (ggf. entpacken und) laden

Falls die CSV-Datei nur gezipped vorliegt:
- führen Sie die folgende Zelle aus, und dann nicht(!) die übernächste

ansonsten
- führen Sie nur die übernächste Zelle aus.

In [ ]:
import zippen

training_data_list = zippen.zip_entpacken ("mnist_train.csv", "mnist_dataset")

In [ ]:
# load the mnist training data CSV file into a list
training_data_file = open("mnist_dataset/mnist_train.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

In [ ]:
training_data_list[0]

## KNN trainieren

In [ ]:
# train the neural network

# epochs is the number of times the training data set is used for training
epochs = 10
print ("Bitte warten!")

for e in range(epochs):
    print(e, "von", epochs, "Epochen gestartet.")
    # go through all records in the training data set
    for record in training_data_list:
        #print(record)
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)
        pass
    pass
print ("Done")

## Alle Trainingsdaten testen

***Zur Kontrolle werden alle Trainingsdaten getestet, da (natürlich) ein KNN in der Regel nicht notwendigerweise alle Trainingsdaten korrekt klassifiziert.***

In [ ]:
def test_all_train_data():
    korrekte = 0
    anzahl = len(training_data_list)
    wrongIndizes = []
    
    for index in range (anzahl):
        
        img_array = np.asfarray(training_data_list[index].split(","))[1:].astype(int)
        img_data  = img_array
        img_data = (img_data / 255.0 * 0.99) + 0.01

        # data is remaining values
        inputs = img_data

        # query the network
        outputs = n.query(inputs)
        
        # the index of the highest value corresponds to the label
        label = numpy.argmax(outputs)
        correct_label = int(training_data_list[index][0])
 
        if (label == correct_label):
            korrekte += 1
        else:
            wrongIndizes.append(index)
    print ("Von den", anzahl, "Trainingsdaten sind", korrekte, "korrekt,", anzahl - korrekte, "falsch zugeordnet")
    # print(wrongIndizes)


In [ ]:
test_all_train_data()

## Alle Testdaten laden

**Bitte die geünschten Testdaten angeben!**

In [ ]:
# our own image test data set
our_own_dataset = []

# load the png image data as test data set
for image_file_name in glob.glob('my_own_images/2828_my_own_?.png'):
#for image_file_name in glob.glob('Meine_Fotos/2828_bov_?.png'):
    
    # use the filename to set the correct label
    label = int(image_file_name[-5:-4])
    
    # load image data from png files into an array
    print ("loading ... ", image_file_name)
    img_array = imageio.imread(image_file_name, as_gray=True)
    
    # reshape from 28x28 to list of 784 values, invert values
    img_data  = 255.0 - img_array.reshape(784)
    
    # then scale data to range from 0.01 to 1.0
    img_data = (img_data / 255.0 * 0.99) + 0.01
    print(numpy.min(img_data))
    print(numpy.max(img_data))
    
    # append label and image data  to test data set
    record = numpy.append(label,img_data)
    our_own_dataset.append(record)
    
    pass

## Einen Testdatensatz testen

In [ ]:
# Nummer des Testbildes:
# record to test
item = 3

In [ ]:
# test the neural network with our own images

# plot image
matplotlib.pyplot.imshow(our_own_dataset[item][1:].reshape(28,28), cmap='Greys', interpolation='None')

# correct answer is first value
correct_label = our_own_dataset[item][0]
# data is remaining values
inputs = our_own_dataset[item][1:]

# query the network
outputs = n.query(inputs)
for i in range (output_nodes):
    print (f'Wahrscheinlichkeit = {outputs[i][0]:1.5f} für die Ziffer {i}')
        

# the index of the highest value corresponds to the label
label = numpy.argmax(outputs)
print("network says ", label)
# append correct or incorrect to list
if (label == correct_label):
    print ("match!")
else:
    print ("no match!, should be", int(correct_label))
    pass


## Alle Testdaten testen

In [ ]:
for item in range (len(our_own_dataset)):
    
    # correct answer is first value
    correct_label = our_own_dataset[item][0]
    # data is remaining values
    inputs = our_own_dataset[item][1:]

    # query the network
    outputs = n.query(inputs)

    # the index of the highest value corresponds to the label
    label = numpy.argmax(outputs)
    print("Bildnummer", item, ": network says:", label, "; shoud be:", int(correct_label))
    # append correct or incorrect to list
    if (label == correct_label):
        print ("match!")
    else:
        print ("no match!")

## Ein eigenes Bild testen

In [ ]:
def testen (image_file_name, correct_label):
    img_array = imageio.imread(image_file_name, as_gray=True)
    
    # reshape from 28x28 to list of 784 values, invert values
    img_data  = 255.0 - img_array.reshape(784)
    
    # then scale data to range from 0.01 to 1.0
    img_data = (img_data / 255.0 * 0.99) + 0.01
    #print(numpy.min(img_data))
    #print(numpy.max(img_data))
        
    # plot image
    matplotlib.pyplot.imshow(img_data.reshape(28,28), cmap='Greys', interpolation='None')

    # data is remaining values
    inputs = img_data

    # query the network
    outputs = n.query(inputs)
    for i in range (output_nodes):
        print (f'Wahrscheinlichkeit {outputs[i][0]:1.5f} für die Ziffer {i}')
        
    
    # the index of the highest value corresponds to the label
    label = numpy.argmax(outputs)
    print("network says ", label)

    # append correct or incorrect to list
    if (label == correct_label):
        print ("match!")
    else:
        print ("no match!, should be", int(correct_label))

    
testen ('Meine_Fotos/2828_bov_2_8.png', 8)
    
 